In [18]:
import pandas as pd
import json
import openai
import numpy as np

# 设置OpenAI API秘钥
openai.api_key = 'YOUR_API_KEY'

In [13]:
# 加载数据集
with open('datasets/qa_dataset_test.json', 'r') as f:
    data = json.load(f)
# 将数据集转化为pandas DataFrame
df = pd.DataFrame(data)

In [14]:
# 利用模型对输入进行预测
import requests

base_url = "http://7801-35-247-191-92.ngrok-free.app"

def predict(input):
    headers = {'Content-Type': 'application/json'}
    data = {'input': input}
    response = requests.post(base_url + '/question-answering', headers=headers, data=json.dumps(data))
    return response.json()['output']

df['result1'] = df['input'].apply(predict)


In [ ]:
df.to_csv("datasets/qa_test_result.csv")
df

In [9]:
df.to_json("datasets/qa_50_result.json")

In [16]:


# 定义GPT-3.5提示词
prompt_template = """
You will be given a conversation between a human and a robot. Each utterance will begin with either "Human: " if said by the human, or "Robot: " if said by the robot.
Your task is to rate the helpfulness of the robot's responses in the given conversation on one metric.
Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:
Helpfulness (1-3) Are the robot's responses helpless/helpful?
- A score of 1 (helpless) means that the responses are meaningless, out of topics, or just safe words. They are helpless to the human.
- A score of 2 (somewaht helpful) means the responses is somewhat helpful and can assist the human to solve a few tasks. However, there could be mistakes on robot's responses.
- A score of 3 (helpful) means that the responses are vert helpful. Robot's responses in the conversation can fulfill most of human's requirements.

Evaluation Steps:
1. Read the conversation, especially the responses by the robot.
2. Rate the robot's responses on a scale of 1-3 for helpfulness, according to the criteria above.
3. Provide a brief explanation for your rating, referring to specific aspects of the response and the conversation.

Conversation:
Human: {input}
Robot: {output}

Evaluation Form (scores ONLY):
- Helpfulness:
"""

# 定义一个函数来生成提示词
def generate_prompt(row):
    return prompt_template.format(input=row['input'], output=row['result1'])

# 生成提示词
df['prompt'] = df.apply(generate_prompt, axis=1)


In [19]:
# 调用GPT-3.5模型进行评分
def get_evaluation(prompt):
    response = openai.Completion.create(engine="text-davinci-003", prompt=prompt, max_tokens=64)
    score = response.choices[0].text.strip()
    return int(score)

df['evaluation'] = df['prompt'].apply(get_evaluation)

# 计算中位数阈值
median_score = np.median(df['evaluation'])

# 判断回答是否得好
df['evaluation_binary'] = df['evaluation'].apply(lambda x: 1 if x > median_score else 0)

# 计算准确率
accuracy = df['evaluation_binary'].sum() / len(df)

print('模型在该数据集的准确率为: ', accuracy)

ValueError: invalid literal for int() with base 10: "3\n\nExplanation:\nThe robot's response is helpful because it provides a code snippet to solve the issue identified by the human - an undefined function call in main.go on line 4. The robot's respons